# Studying the orbit of satellite galaxies

The purpose of this tutorial is to help *you* with modelling and studying the orbit of galaxies.
There are a few things you need to consider when creating your model.
- Firstly: what data are you using? When you start your model, you need to have some data on the current 6D properties of your galaxy. The two main methods (that may either produce the same or different results) are 1. manually inputting the 6D data as a list, or 2. using the orbit.Orbit.from_name method. The from_name method imports the data from the SIMBAD database automatically.
- Next: what potential are you using for the Milky Way? the galpy package's default potential is 'MWPotential2014', from *Include the source*, however it also includes others like McMillan17, or "". You can also instantiate the potential from scratch, using different potential objects like "".
- And: Do you want to include the LMC potential? In recent studies *cite?*, it's been found that the LMC's effects on the Milky Way are fairly substantial, and depending on the distance from the Milky Way's centre, it's inclusion will affect how satellite galaxies orbit the Milky Way. You can either include it by making a simplified potential, ..... Or, you can include its affects on the acceleration of the Milky Way galactic centre, by calculating the resulting force of the LMC on the MW.
- Lastly: For what timeframe do you want to model your orbit? You can integrate your orbit forwards, backwards, or even both to compare the past modelled orbit to the predicted future orbit. You can also choose exactly how far back (or far forwards) you want to integrate the orbit, just keep in mind that the further you model from the present, the less accurate the model may become.